set up env variables (e.g. in a .env file if using vscode)

- "KEY", for your openai API key
- "PROMPT_SYSTEM" (optionally) if you want to give your chatbot a default personality (e.g. helpful expert in area x)

In [ ]:
#dependencies

#%pip install openai
#%pip install ipywidgets

In [14]:

import os
import openai
import ipywidgets as widgets
from IPython.display import display

#api key
openai.api_key = os.getenv("KEY")
#default personality / behavior
default_system = os.getenv("PROMPT_SYSTEM") 

if (default_system is not None):
    #initializing chat history with the system prompt
    conversation = [{"role": "system", "content": default_system}]
else:
    conversation = []
    
#the maximum context window (short term memory) is 4096 tokens, so we need to keep track of token usage to limit the risk running into an API error; the webUI is using a sliding window to manage around this limit
token_count = 0


#API calls to OpenAI
def get_gpt (prompt):
    global token_count
    global conversation

    #if our current token_count is above 3000 then we remove the oldest message from the conversation, if there is a personality set up in default_system then the 2nd oldest
    if token_count > 3000:
        if (default_system is not None):
            conversation.pop(1)
        else:
            conversation.pop(0)

    conversation.append({"role": "user", "content": prompt})
    response = openai.ChatCompletion.create(
            #gpt-3.5-turbo is the current equivalent of what's behind the web UI
            model="gpt-3.5-turbo", 
            messages=conversation,
            #the higher the temperature the more creative / diverse the responses will be, minimum is 0 (deterministic), maximum is currently 2 (too much)
            temperature = 0.3,
            max_tokens = 1000,
            n = 1            
        )
    message = response.choices[0].message.content
    token_count = response.usage.total_tokens

    conversation.append({"role": "assistant", "content": message})
    return response



In [15]:
from IPython.display import HTML
import ipywidgets as widgets

html_template = """
<div>
    <pre>
        <div>{}</div>
        <div>{}</div>
    </pre>
</div>
"""

#displaying messages
def render_message(message):
    if message["role"] == "assistant":
        return(f"<i>ChatGPT</i>: {message['content']}")
    else:
        return(f"<b>You</b>: {message['content']}")
    

# Define a function to handle user input
def handle_input(button):
    user_input = input_widget.value
    output.append_display_data("--------------message sent--------------")
    input_widget.value = ''
    
    response = get_gpt(user_input)
    latest_messages = html_template.format(render_message(conversation[-2]),render_message(conversation[-1]))
    output.append_display_data(HTML(latest_messages))

    print('current token count: ', token_count)
    
def get_bigger(args):        
    input_widget.rows = input_widget.value.count('\n') + 4

output = widgets.Output(layout={'border': '1px solid black'})

input_widget = widgets.Textarea(
    value='',
    placeholder='Type your message here...',
    description='Input:',
    disabled=False,
    layout={'width': '100%', 'height': 'auto'},
    rows = 5
)

input_widget.observe(get_bigger,'value')

# Create a button to send the input
send_button = widgets.Button(description='Send', layout={'width': '100%'})
send_button.on_click(handle_input)

# Display the input widget and send button
display(widgets.VBox([output,input_widget,send_button]))



current token count:  111
current token count:  818
